In [1]:
from Levenshtein import distance as levenshtein_distance
from time import *
import time
import numpy as np
import math
import random
import os
import pandas as pd

# Division Kmer

In [14]:
def kmers_split(seq, k, nb_indice):
    """
    Split kmers
    Input1 seq      : sequence to split
    Input2 k        : length of kmers
    Input3 nb_indice: Number of kmer with same index
    Output list_    : list containing a list of kmer with their indexes 
    """
    list_ = []
    
    # on divise nos k-mers en sous partie. le but est de comparer tous les k-mers avec le meme indice=div_counter
    # dans la sequence query avec tous les k-mers avec le meme indice dans la seq ref
    # cela permettra d eviter a comparer les k-mers en premier avec des k-mers en dernier
    
    div = nb_indice
    div_counter = 0
    
    for i in range(len(seq)-(k-1)): 
        if i == div:
            div_counter += 1
            div += nb_indice
        list_.append([seq[i:i+k], div_counter])   
        #list_.append(seq[i:i+k])
    return list_

In [18]:
seqte = 'CGACAAACTTGCGAGTA'
kmers_split(seqte, 3, 3)

[['CGA', 0],
 ['GAC', 0],
 ['ACA', 0],
 ['CAA', 1],
 ['AAA', 1],
 ['AAC', 1],
 ['ACT', 2],
 ['CTT', 2],
 ['TTG', 2],
 ['TGC', 3],
 ['GCG', 3],
 ['CGA', 3],
 ['GAG', 4],
 ['AGT', 4],
 ['GTA', 4]]

In [ ]:
[['CGA', 0], ['GAC', 0], ['ACA', 0], ['CAA', 1], ['AAA', 1], ['AAC', 1], ['ACT', 2], ['CTT', 2], ['TTG', 2], ['TGC', 3], ['GCG', 3], ['CGA', 3], ['GAG', 4], ['AGT', 4], ['GTA', 4]]

In [19]:
seqte2 = 'CGACAAACTTCTTACGA'
kmers_split(seqte2, 3, 3)

[['CGA', 0],
 ['GAC', 0],
 ['ACA', 0],
 ['CAA', 1],
 ['AAA', 1],
 ['AAC', 1],
 ['ACT', 2],
 ['CTT', 2],
 ['TTC', 2],
 ['TCT', 3],
 ['CTT', 3],
 ['TTA', 3],
 ['TAC', 4],
 ['ACG', 4],
 ['CGA', 4]]

In [ ]:
[['CGA', 0], ['GAC', 0], ['ACA', 0], ['CAA', 1], ['AAA', 1], ['AAC', 1], ['ACT', 2], ['CTT', 2], ['TTC', 2], ['TCT', 3], ['CTT', 3], ['TTA', 3], ['TAC', 4], ['ACG', 4], ['CGA', 4]]

In [41]:
# test
k = 5
taille_v = 280
nb_indice = 4
query_s2 ="CAGGGACAGCTGGTGCAGTCTGGGGCTGAGGAGAAGAGGCCTGGGGCCTCAGTGAAAGTCTCCTGCAAGGCCTCTGGATACACCATCACCAGCTACTGTATGCACTGGGTCCACCAGGTCCATGCACAAGGGCTTGAGTGGATGGGATTGGTGTGCCCTAGTGATGGCAGCACAAGCTATGCACAGAAGTTCCAGGCCAGAGTCACCATAACCAGGGACACATCCATGAGCACAGCCTACATGGAGCTAAGCAGTCTGAGATCTGAGGACACGGCCATGTATTACTGTGTGAGACGCCAGCTATGGGAGACTGATTGCTTCCAGCACTGGGGCCAGGTCACCCTGGTCACTGTCTCCTCAG"

#kmers_split("CGACAAACGTCG", k, nb_indice )
#resultats attendus
#[['CGACA', 0],
# ['GACAA', 0],
# ['ACAAA', 0],
# ['CAAAC', 0],
# ['AAACG', 1],
# ['AACGT', 1],
# ['ACGTC', 1],
# ['CGTCG', 1]]

In [4]:
#kmers_split("CGACAAAC", 3, 2 )

#### 1er petit Test sur les données

In [5]:
query_s2 ="CAGGGACAGCTGGTGCAGTCTGGGGCTGAGGAGAAGAGGCCTGGGGCCTCAGTGAAAGTCTCCTGCAAGGCCTCTGGATACACCATCACCAGCTACTGTATGCACTGGGTCCACCAGGTCCATGCACAAGGGCTTGAGTGGATGGGATTGGTGTGCCCTAGTGATGGCAGCACAAGCTATGCACAGAAGTTCCAGGCCAGAGTCACCATAACCAGGGACACATCCATGAGCACAGCCTACATGGAGCTAAGCAGTCTGAGATCTGAGGACACGGCCATGTATTACTGTGTGAGACGCCAGCTATGGGAGACTGATTGCTTCCAGCACTGGGGCCAGGTCACCCTGGTCACTGTCTCCTCAG"
ref_v1_or21 = "caggtacagctggtgcagtctggggctgaggtgaagaagcctggggcctcagtgaaggtctcctgcaaggcttctggatacaccatcaccagctactgtatgcactgggtgcaccaggtccatgcacaagggcttgagtggatgggattggtgtgccctagtgatggcagcacaagctatgcacagaagttccaggccagagtcaccataaccagggacacatccatgagcacagcctacatggagctaagcagtctgagatctgaggacacggccatgtattactgtgtgagaga"
ref_v1_18_01 = "caggttcagctggtgcagtctggagctgaggtgaagaagcctggggcctcagtgaaggtctcctgcaaggcttctggttacacctttaccagctatggtatcagctgggtgcgacaggcccctggacaagggcttgagtggatgggatggatcagcgcttacaatggtaacacaaactatgcacagaagctccagggcagagtcaccatgaccacagacacatccacgagcacagcctacatggagctgaggagcctgagatctgacgacacggccgtgtattactgtgcgagaga"

In [6]:
r_kmer = kmers_split(ref_v1_or21.upper(), 5, nb_indice )
r_kmer2 = kmers_split(ref_v1_18_01.upper(), 5, nb_indice)
q_kmer = kmers_split(query_s2, 5, nb_indice)
#print(r_kmer)

# Comparaison des listes de kmer de la forme [[kmer,indice]]

In [35]:
import time
def comparaison(query_kmer, ref_kmer, bloc_max, k=5):
    """
    Comparing two different lists of [kmer, indice]
    Input1 query_kmer: list of [kmer, indice] of the query sequence
    Input2 ref_kmer  : list of [kmer, indice] of the ref sequence
    Input3 bloc_max  : stops the algorithm if there is no identical [kmer, indice] for bloc_max times
    Input4 k         : length of kmers
    Output1 count    : number of similar [kmer, indice] between the two sequences
    Output2 position : last nucleotid position of the comparaison     
    """
    count = 0 # pour le nombre de [kmer, indice] identique dans les deux sequences
    block = 0 # pour areter l'algorithme si pour un nombre 'block' successive on n'a pas trouver des [kmer, indice] identique
    query_kmer_copy = query_kmer.copy()
    last_pos_query = 0 # dernier element de query qui a ete comparé afin de le renvoyer si on atteint bloc_max
    position = 0
    # par exemple si l algo atteint le bloc_max, tous les [kmer, indice] entre block = 0 et bloc_max
    # n'ont pas trouvé un autre [kmer, indice] identique à eux. Ces [kmer, indice] doivent donc etre reutilise
    # pour d'autres parties. Si on etudier les V alors ces [kmer, indice] doivent etre utilise pour les D
    # C'est ainsi que last_pos_query fait en sorte de respecter cela.

    for kmer in ref_kmer: # on parcout les [kmer, indice] de la sequence reference
        if kmer in query_kmer_copy: # on voit si ces [kemr, indice] sont dans la sequence query
            count += 1 # on precise qu'on a trouve un [kmer, indice]
            query_kmer_copy.remove(kmer)
            last_pos_query = query_kmer.index(kmer)
            
            block = 0 # on remet a 0 le block vu qu'on a trouver un [kmer, indice] 

        #juste pour le rapport, a enlever apres
        #if k + len(query_kmer[0:last_pos_query]) == 280:
         #   #print("ici")
        #    break
            
        if block == bloc_max: # si bloc_max est atteint
            position = k + len(query_kmer[0:last_pos_query]) # on prend la position finale de la sequnce etudier
            # suivant la formule suivante.
            # Explication formule:
            # Soit les données suivantes: "CGACTA", k = 3, nbre_indice = 2
            # kmer_indice = [['CGA', 0], ['GAC', 0], ['ACT', 1], ['CTA', 1]]
            # Si on veut calculer la position du 5eme nucleotide par exemple et la derniere postion query
            # est ['ACA', 1] alors
            # position = 3 + kmer_indice[0:3] = 5
            
            
            return count, position
        
        block += 1
    
    # si on atteint jamais le block_max alors on prend tout
    index_dernier_element = query_kmer.index(query_kmer[-1])
    position = k + len(query_kmer[0:index_dernier_element])
    
    return count, position
debut = time.time()
comparaison(q_kmer, r_kmer, 40)
comparaison(q_kmer, r_kmer, 40)
comparaison(q_kmer, r_kmer, 40)
fin = time.time()
#print("temps:", fin-debut)

In [8]:
#comparaison(q_kmer, r_kmer, 40)

# Resultat attendu
# (260, 361)

#### 2eme petit Test sur les données

In [9]:
query_IGHV3_30_3_01 = "CAAGTGCAGCTGGTGGAGTCTGGGGGAGGCGTGGTCCAGCCTGGGAGGTCCCTGAGACTCTCCTGTGCAGCCTCTGGATTCACCTTCAATAGCAATGCTATGCACTGGGTCCGCCAGGCTCCAGGCAAGGGGCTGGAGTGGGTGGCAGTTATATCATATGATGGAAGAAATAGATACTACGCAGACTCCGTGAAGGGCCGATTCACCATCTCCAGAGACAATTCCAAGAACACGCTGTATCTGCAAATGAACAGCCTGAGAGCTGAGGACACGGCTGTGTATTACTGTGCGAGCGTTAAATGTGGATATAGTGGCTACGATTACACATACTTCCAGCACTGGGGCCAGGGCACCCTGGTCACCGTCTCCTCAG"
ref_IGHV3_30_3_01 = "caggtgcagctggtggagtctgggggaggcgtggtccagcctgggaggtccctgagactctcctgtgcagcctctggattcaccttcagtagctatgctatgcactgggtccgccaggctccaggcaaggggctggagtgggtggcagttatatcatatgatggaagcaataaatactacgcagactccgtgaagggccgattcaccatctccagagacaattccaagaacacgctgtatctgcaaatgaacagcctgagagctgaggacacggctgtgtattactgtgcgagaga"
ref_IGHV3_30_04 = "caggtgcagctggtggagtctgggggaggcgtggtccagcctgggaggtccctgagactctcctgtgcagcctctggattcaccttcagtagctatgctatgcactgggtccgccaggctccaggcaaggggctggagtgggtggcagttatatcatatgatggaagtaataaatactacgcagactccgtgaagggccgattcaccatctccagagacaattccaagaacacgctgtatctgcaaatgaacagcctgagagctgaggacacggctgtgtattactgtgcgagaga"
# test
k = 5
nb_indice = 7
bloc_max = 40

q_kmer2 = kmers_split(query_IGHV3_30_3_01, k, nb_indice )
r_kmer2 = kmers_split(ref_IGHV3_30_3_01.upper(), k, nb_indice)
r_kmer22 = kmers_split(ref_IGHV3_30_04.upper(), k, nb_indice)

#print(comparaison(q_kmer2, r_kmer2, bloc_max))
#print('///')
#print(comparaison(q_kmer2, r_kmer22, bloc_max))

# Resultat attendu
# (266, 373)
# (266, 373)

# Etude sur nos Fichiers

In [10]:
# lecture du fichier reference
def read_db(path):
    """
    Reading of the reference genes
    Input path    : path of the file
    Output dic_db : dictionary with name of the sequence as key and the sequence itself as value
    """
    dic_db = {}
    f = open(path, 'r')
    while 1:
        line = f.readline().strip()
        if line == '':
            break
        if line[0] == '>':
            fa = f.readline().strip()
            fa = fa.upper()
            dic_db[line[1:]] = fa
    f.close()
    return dic_db

In [11]:
# extraction des genes ref
db_path = "human_germline//IGHV.fasta"
dic_db = dict(read_db(db_path))
#print(dic_db['IGHV1-18*01'])

In [12]:
## extraction des genes query
# ouvrir le fichier query et mettre tous les elements v dans un dictionnaire ayant comme cle nom du v 
# et comme valeur la sequence
# On fait pareil pour les D et J. On aura a la fin trois listes de la forme: 
# l1_keys   = [v1, v2, v3]
# l1_values = [seq1, seq2, seq3]
# l2_keys   = [d1, d2, d3]
# l2_values = [seq1, seq2, seq3]
# l3_keys   = [j1, j2, j3]
# l3_values = [seq1, seq2, seq3]

def extract_vquery(fichier):
    """
    Extracting the query sequence for the V, D and J
    Input1 fichier      : path of the file
    Ouput1 vquery_keys  : name of the V
    Ouput2 vquery_values: sequence of the V
    Ouput3 dquery_keys  : name of the D
    Ouput4 dquery_values: sequence of the D
    Ouput5 jquery_keys  : name of the J
    Ouput6 jquery_values: sequence of the J
    Ouput7 l_taille     : list containaing the length of each seq
    
    """
    
    name_fichier_query = fichier
    f = open(name_fichier_query,'r')
    i = 0
    #extraire les v
    vquery_keys = []
    vquery_values = []
    
    #extraire les d
    dquery_keys = []
    dquery_values = []
    
    #extraire les j
    jquery_keys = []
    jquery_values = []
    
    #extraire la taille
    l_taille = []
    
    while True:
        # lecture du fichier query
        line = f.readline().strip()
        
        if line == '':
            break
            
        # extraction des sequences ainsi que le nom du bloc D
        li = line
        if li[0] == '>':
            tmpv = li.split('|')[1]
            tmpd = li.split('|')[2]
            tmpj = li.split('|')[3]

            vquery_keys.append(tmpv)
            dquery_keys.append(tmpd)
            jquery_keys.append(tmpj)
            continue
        
        # on mets la meme sequence pour les trois listes
        vquery_values.append(li.upper())
        dquery_values.append(li.upper())
        jquery_values.append(li.upper())
        l_taille.append(len(li)) # taille de la liste
        i += 1
    f.close()
        
    return vquery_keys, vquery_values, dquery_keys, dquery_values, jquery_keys, jquery_values, l_taille

In [13]:
#name_fichier_vquery = 'VDJ-Kernel-main//data//datasets//simulations//simple_plus_indels_40Mut_out.fasta'
#vquery_keys, vquery_values, dquery_keys, dquery_values, jquery_keys, jquery_values, l_taille = extract_vquery(name_fichier_vquery)

#print(vquery_keys)
#print(vquery_values[0:2])
#print(l_taille)

In [14]:
def compare_fichier(dic_db_keys, dic_db_values, q_kmer, bloc_max, nb_indice, k):
    """
    Comparing the [kmer, indice] of one query sequence with all the [kmer, indice] references
    Input1 dic_db_keys  : keys of the reference sequences
    Input2 dic_db_values: values of the reference sequence
    Input3 q_kmer       : list of [kmer, indice] for query sequence
    Input4 bloc_max     : stops the algorithm if there is no identical [kmer, indice] for bloc_max times
    Input5 nb_indice    : Number of kmer with same index
    Input6 k            : length of kmers
    Output1 dic         : dictionary as the name of the query as key and the number of similar [kmer, indice] between the two sequences as value
    Output2 dic_pos     : dictionary as the name of the query as key and last nucleotid position of the comparaison as value
    
    """
    dic = {}
    dic_pos = {}
    for j in range(len(dic_db_keys)):
        r_kmer = kmers_split(dic_db_values[j], k, nb_indice) # on divise en [kmer, indice] la seq ref
        comp, position = comparaison(r_kmer, q_kmer, bloc_max ) # on realise la comparaison
        cle = dic_db_keys[j]
        dic[cle] = comp
        dic_pos[cle] = position
            
    return dic, dic_pos

In [15]:
# dans le cas où notre algo a eu 2 ou plus sequences de meme score, on fait un test de plus, qui est celui de
# levenshtein et on reduit le nom des sequences en enlevant les allèles. On considere ainsi que
# 'IGHV3-30*04', 'IGHV3-30*09' sont identiques
def ref_in_listoOfKeys(listOfKeys, listOfRefs, query):
    """
    Using levenshtein if the alogirthm found 1 or more sequences with the same score
    Input1 lisOfKeys  : keys of the same score
    Input2 query      : 
    Output1 dic_count :
    Output2 cle       :
    Output3 cle2      :
    """
    dict_count = {}
    cle = ''
    cle2 = ''
    for i in range(len(listOfRefs)):
        distance = levenshtein_distance(listOfRefs[i], query )
        ref = listOfKeys[i]
        
        try:
            ref = listOfKeys[i]
            pos = ref.index('*')
            cle = ref[:pos] # on enleve le '*' de la seq et tout ce qui est apres le '*'
            if cle[-1] == 'D':
                cle = cle[:-1]
            cle2 = ref
            if cle in dict_count:
                dict_count[cle] += 1
            else:
                dict_count[cle] = 0
        except:
            ref = listOfKeys[i]
            pos = len(ref)
            cle = ref[:pos]
            if cle[-1] == 'D':
                cle = cle[:-1]
            cle2 = ref
            dict_count[cle] = 0
    return dict_count, cle, cle2



In [16]:
# meme fonction qu'avant mais on calcul en plus la distance
def ref_in_listoOfKeys_distance(listOfKeys, listOfRefs, query, distance_max):
    dict_count = {}
    for i in range (len(listOfKeys)):
        distance = levenshtein_distance(listOfRefs[i], query )
        #print(distance)
        
        try:
            ref = listOfKeys[i]
            pos = ref.index('*')
            cle = ref[:pos]
            if cle[-1] == 'D':
                cle = cle[:-1]
            cle2 = ref
            if cle in dict_count:
                dict_count[cle] += 1
            else:
                dict_count[cle] = 0
        except:
            ref = listOfKeys[i]
            pos = len(ref)
            cle = ref[:pos]
            if cle[-1] == 'D':
                cle = cle[:-1]
            cle2 = ref
            dict_count[cle] = 0

        if distance > distance_max:
            distance_max = 0
            max_dic = cle
            max_dic_avec_cle = cle2
    return dict_count, max_dic, cle, max_dic_avec_cle

In [17]:
# fonction pour enlever le '*' de la seq ref et tout ce qui est apres le '*'
def remove_etoile(tmp_d):
    try:
        pos = tmp_d.index('*')
        tmp_d = tmp_d[:pos]
    except:
        tmp_d = tmp_d
    return tmp_d

## Partie V

In [18]:
import time
def main_codeV(dic_db, vquery_keys, vquery_values, nb_indice, bloc_max, i, while_i):

    #debut_algo = time.time()
    # gene ref
    dic_db_keys = list(dic_db.keys()) # on prend le nom du gene ref
    dic_db_values = list(dic_db.values()) # one prend la sequence du gene ref
    
    
    dic = {}
    tot_seq = 0 # le nombre de seqeunces etudiees
    tot_vrai = 0 # le nombre de vrais positifs
    temps_max = 0
    temps_levenshtein = 0
    
    l_pos_max = []
    
    # query
    #print(len(vquery_keys))
    #print(len(vquery_values))
    k = 5
    taille_V = 280
    while i < while_i:
        
        tmpv = vquery_keys[i]
        query = vquery_values[i]
        q_kmer = kmers_split(query, k, nb_indice) # on divise en [kmer, indice] la seq query

        debut_comp = time.time()
        
        # on commence l'algorithme
        dic, dic_pos = compare_fichier(dic_db_keys, dic_db_values, q_kmer, bloc_max, nb_indice, k)
        fin_comp = time.time()
        temps_comp = fin_comp - debut_comp
        
        #print('list position', dic_pos)
        
        max_ = max(dic.items(), key=lambda x: x[1]) # on prend le score max des comparaisons faites
        listOfKeys = list()
        listOfRefs = list()
        #print(max_)
        # on teste s'il y a plusieurs gene ref qui ont le meme score et on les mets dans une liste
        for key, value in dic.items():
            if value == max_[1]:
                #print(dic_db[key])
                #listOfKeys.append(key)
                
                listOfKeys.append(key)
                listOfRefs.append(dic_db[key])
                
        distance = 0
        distance_max = 0
        max_dic = ''
        
        # s'il y a bien plusieurs gene ref dans la liste alors on utilse la fct levenshtein_distance pour
        # essayer de trouver la bonne sequence
        dict_count = {} # pour compter le nombre de fois où on a le même gene d allèles differents
        #print(query)
        dict_count, cle, cle2 = ref_in_listoOfKeys(listOfKeys, listOfRefs, query)
        #print(listOfKeys)

   
        if len(listOfKeys) == 1:
            temps_levenshtein = 0
            
            if cle[-1] == 'D': # on elenve les duplicer
                max_dic = cle[:-1]
            else:
                max_dic = cle
            max_dic_avec_cle = cle2
        
        if len(listOfKeys) > 1 :
            debut_levenshtein = time.time()
            dict_count, max_dic, cle, max_dic_avec_cle = ref_in_listoOfKeys_distance(listOfKeys, listOfRefs, query, distance_max)
                    
            fin_levenshtein = time.time()
            temps_levenshtein = fin_levenshtein - debut_levenshtein
        # on enleve le '*' de la seq ref et tout ce qui est apres le '*
        
        #dic_pos_max[max_dic_avec_cle] = dic_pos[max_dic_avec_cle]
        l_pos_max.append(dic_pos[max_dic_avec_cle])
        #print("dic_pos[max_dic]",dic_pos[max_dic_avec_cle])
        tmpv = remove_etoile(tmpv)
        
        if tmpv[-1] == 'D': # on elenve les duplicer
            tmpv = tmpv[:-1]
        
        
        print("Sequences avec même score :", listOfKeys)
        print("Nombre d'iteration        :", i)
        print("Sequence reference        :", max_dic)
        #print("Sequence reference        :", max_dic_avec_cle)
        print("Sequence query            :", tmpv)
        print("Temps levenshtein         :", temps_levenshtein)
        print("Temps complet:            :", temps_comp)
        
        temps_max += temps_comp
            
        
        # on test si on a des vrais positifs
        if max_dic == tmpv: # si c'est des  vrais positifs
            print("-> VRAI Même sequence trouvée")
            tot_vrai += 1
        else:
            #tttt = 0
            print("-> FAUX Différente sequence trouvée")
        print("---------------------------------------------------------------------------------")
        
        dic = {}
        tot_seq += 1
        i += 1
    vrai_pos = (tot_vrai/tot_seq)*100
    
    m, s = divmod(int(temps_max), 60)
    h, m = divmod(m, 60)

    
    
    print("                                 Résultat final                                  ")
    print("---------------------------------------------------------------------------------")
    print("Nom du fichier ref                         :", db_path)
    print("Nom du fichier query                       :", name_fichier_vquery[46:len(name_fichier_vquery)])
    print("Nombre de vrais positifs                   :", tot_vrai)
    print("Nombre de séquences testées                :", tot_seq)
    print("Pourcentage de vrais positifs pour les V   :", vrai_pos, "%")
    #print("Temps final de l'algorithme                :", strftime('%H %M %S', gmtime(temps_max)))
    print("Temps final de l'algorithme                :", f'{h:d}:{m:02d}:{s:02d}')
    print("---------------------------------------------------------------------------------")
    print("---------------------------------------------------------------------------------")
    print(l_pos_max)
    
    
    return l_pos_max
    #return vrai_pos, temps_max

In [19]:
#print(l_taille)

In [20]:
#print(l_finalV)

#### Tests pour determiner le meilleur indice et le meilleur block_max

In [21]:
#df = pd.DataFrame(data, index = index)
#print (df)

In [22]:
#nombres indices: 40 bloc_max: 20 vrai_pos: 88.0 temps: 7.739743947982788
#nombres indices: 40 bloc_max: 25 vrai_pos: 92.0 temps: 8.916382789611816
#nombres indices: 40 bloc_max: 30 vrai_pos: 96.0 temps: 10.858557224273682
#nombres indices: 40 bloc_max: 35 vrai_pos: 96.0 temps: 15.681548118591309

In [23]:
#BEST
#main_codeV(dic_db, vquery_keys, vquery_values, 40, 30, i, 100)

In [24]:
#main_codeV(dic_db, vquery_keys, vquery_values, 40, 30, i, 1000)

In [25]:
#main_codeV(dic_db, vquery_keys, vquery_values, 40, 25, i, 1000)

In [26]:
#main_codeV(dic_db, vquery_keys, vquery_values, 50, 30, i, 100)

dic_db, vquery_keys, vquery_values, nb_indice, bloc_max, i, while_i

def test(while_i):
    # on veut essayer de trouver le nb_indices et bloc_max les plus optimaux
    nb_indices = 30
    bloc_max = 20
    while_i = 25
    for ni in range(5):
        for bm in range(5):
            
            vrai_pos, temps = main_codeV(dic_db, vquery_keys, vquery_values, nb_indices, bloc_max, 0, while_i)
            print("nombres indices:", nb_indices, "bloc_max:", bloc_max, "vrai_pos:", vrai_pos, "temps:", temps)
            bloc_max += 5
            
        nb_indices += 5
        bloc_max = 35
        
    return 0
#test(50)